In [30]:
import numpy as np

grav = 9.806
grav = 9.806
Cp   = 1004.
Rgas = 287.04

#-----------------------------------------------------------------------------
# subroutines for grid initialization
#
def zgrid(nz, height = 20000., stag=False): 
    dz = height / float(nz)
    if stag:
        dz = height / float(nz-1)
        return np.arange(nz)*dz
    else:
        return (np.arange(nz)+0.5)*dz

#-----------------------------------------------------------------------------
#
def sounding(gz, theta_sfc = 300., qv_sfc = 0.001):
    
    #define parameters
    z_trop     = 12000.
    theta_trop = 343.
    temp_trop  = 213.
    
    # compute theta profile
    theta = np.zeros_like(gz)
    t1    = theta_sfc + (theta_trop - theta_sfc)*((gz/z_trop)**1.25)
    t2    = theta_trop * np.exp(grav * (gz - z_trop) / (Cp * temp_trop))   
    theta = np.where(gz <= z_trop, t1, t2)
    
    # compute rh profile
    rh    = np.where(gz <= z_trop, 1.0 - 0.75*((gz/z_trop)**1.25), 0.25)
    
    # integrate hydrostatic eq
    pi    = np.zeros_like(gz)
    p     = np.zeros_like(gz)
        
    if gz[0] > 1.0:
        dz = np.zeros_like(gz)
        dz[0] = 2.0*gz[0]
        for n in np.arange(1, gz.shape[0] - 1):
            dz[n] = 0.5*(gz[n+1] - gz[n-1])
    
    if gz[0] > 1.:
        pi[0] = 1.0 - 0.5*dz[0]*grav/(Cp*theta_sfc)
    
        for k in np.arange(1,gz.shape[0]):
            pi[k] = pi[k-1] - 2.0*dz[k]*grav/(Cp*(theta[k-1]+theta[k]))
    else:
        pi[0] = 1.0
        for k in np.arange(1,gz.shape[0]):
            pi[k] = pi[k-1] - 2.0*dz[k-1]*grav/(Cp*(theta[k-1]+theta[k]))
    
    # compute pressure (Pa) profile
    p = 1.0e5 * pi**(3.5088)
    t = pi*theta - 273.16
    
    # compute qv profile
    qvs = (380./p) * np.exp(17.27*((pi*theta - 273.16)/(pi*theta-36.)))
    qv  = qvs*rh
    qv  = np.where( qv > qv_sfc/1000., qv_sfc/1000., qv)
    
    # recompute RH so that it reflects the new qv_sfc limit - needed to dewpoint computation.
    rh  = qv/qvs
    
    ess = 6.112 * np.exp(17.67 * t / (t + 243.5))
    val = np.log(rh * ess/6.112)
    td  = 243.5 * val / (17.67 - val)
    
    den = p / (t+273.16 * Rgas)
    
    return pi, theta, qv, p, t, td, den

#-----------------------------------------------------------------------------
#
def shear(gz, type=1, shear=12.5, depth=2500.):

    if type == 0.0:
        return np.zeros_like(gz), np.zeros_like(gz)

    if type == 1:  # 1D WK squall line or supercell shear...

        scale = gz / depth

        u = np.where(scale <= 1.0, shear*scale, shear)

        return u, np.zeros_like(gz)

    if type == 2:  # 2D WK squall line or supercell shear...

        depth = 5000.
        scale = gz / depth
        ugnd  = shear / np.pi
        angle = np.deg2rad(180.0 - (1.0 - scale))
        u     = np.where(scale <= 1.0, ugnd*np.cos(angle), ugnd)
        v     = np.where(scale <= 1.0, ugnd*np.sin(angle),  0.0)

        return u, v

    if type == 3:  # Quarter circle shear

        depth = 2500.
        scale = gz / depth
        vmag  = 8.5

        angle = np.deg2rad(180.0 - 90.0*scale)

        u     = np.where(scale <= 1.0, vmag*np.cos(angle), 0.006*(gz-3000.) )
        v     = np.where(scale <= 1.0, vmag*np.sin(angle), vmag)

        return np.where(u <= 24., u, 24.), v-0.5*vmag
        
#-----------------------------------------------------------------------------
#
def write_2_file(p0, th0, q0, z, theta, qv, u, v, filename='wk.sounding'):
    
    print(filename)
    
    with open(filename, 'w') as f:
        f.write("%12.4f %12.4f  %12.4f \n" % (p0, th0, q0))
        for k in np.arange(z.shape[0]):
            f.write("%12.4f  %12.4f  %12.4f  %12.4f  %12.4f\n" % (z[k], theta[k], 1000.*qv[k], u[k], v[k]))
        
    f.close()


In [27]:
#
# Parameters for squall line soundings for FV3 tests
#

# Create grid
gze = zgrid(301, height = 30000., stag=True)
gzc = zgrid(300, height = 30000.)
dz  = gze[1:] - gze[:-1]

# create soundings

shear0 = 12.5

u, v = shear(gze, shear = 24, type=1)

qv0 = [11.0,12.0,13.0,14.0,15.0,16.0]

for q in qv0:
    
    pi, theta, qv, p, t, td, den = sounding(gze, dz, theta_sfc=300.5, qv_sfc=q)
    
    write_2_file(1000., 300.5, q, gze, theta, qv, u, v, filename = "input_sounding_qv%2.2i_us%4.1f" % (q, shear0))


TypeError: sounding() got multiple values for argument 'theta_sfc'

In [28]:
#
# Parameters for warm thermal with 500 mb tropopause.
#


gze = zgrid(301, height = 30000., stag=True)
gzc = zgrid(300, height = 30000.)
dz  = gze[1:] - gze[:-1]

# create soundings

shear0 = 0.0

u, v = shear(gze, shear = shear0, type=0)

pi, theta, qv, p, t, td, den = sounding(gze, dz, theta_sfc=300.)

write_2_file(1000., 300., 0.001, gze, theta, qv, u, v, filename = "bubble.sound" )



TypeError: sounding() got multiple values for argument 'theta_sfc'

In [32]:
# read in sounding heights for WRF

# path = "/work/wicker/Odin_scr/cm1r20.3/run/Soundings/input_grid_C2500"

# with open(path, 'r') as f:
#     strings = f.readlines()

zh = [0.01073497, 0.04024141, 0.08258476, 0.1361145, 0.2023856, 0.2813303,
    0.372868, 0.4769054, 0.5933367, 0.7220436, 0.8628909, 1.015673, 1.180161,
    1.356162, 1.543466, 1.740841, 1.945226, 2.154314, 2.368158, 2.587233,
    2.811645, 3.0415, 3.276904, 3.517966, 3.764791, 4.017487, 4.276159,
    4.540912, 4.811849, 5.08907, 5.372672, 5.662747, 5.959383, 6.262661,
    6.572655, 6.889427, 7.213032, 7.543508, 7.880885, 8.225174, 8.576372,
    8.934463, 9.299411, 9.671172, 10.0497, 10.43492, 10.82679, 11.22524,
    11.63023, 12.0424, 12.46476, 12.90181, 13.36197, 13.86196, 14.42729,
    15.09925, 15.95271, 17.11467, 18.66372, 20.95429]
    
zhgts = np.asarray([1000.*float(i) for i in zh])

#zhgts = zgrid(300, height = 30000.)

pi, theta, qv, p, t, td, den  = sounding(zhgts, theta_sfc = 300., qv_sfc = 16.0)

u, v = shear(zhgts, type=3, shear=0.006)

for n in np.arange(len(u)):
    print( "%d.  %10.4f.  %10.4f   %10.4f" % (n, zhgts[n], u[n], v[n] ))

    
write_2_file(1000., 300., 16., zhgts, theta, qv, u, v, filename='weisman_quart_qv16.snd')

0.     10.7350.     -8.4998      -4.1927
1.     40.2414.     -8.4973      -4.0351
2.     82.5848.     -8.4886      -3.8091
3.    136.1145.     -8.4689      -3.5239
4.    202.3856.     -8.4314      -3.1720
5.    281.3303.     -8.3676      -2.7553
6.    372.8680.     -8.2678      -2.2768
7.    476.9054.     -8.1212      -1.7409
8.    593.3367.     -7.9161      -1.1541
9.    722.0436.     -7.6402      -0.5247
10.    862.8909.     -7.2810       0.1360
11.   1015.6730.     -6.8271       0.8136
12.   1180.1610.     -6.2683       1.4910
13.   1356.1620.     -5.5964       2.1477
14.   1543.4660.     -4.8065       2.7605
15.   1740.8410.     -3.9024       3.3012
16.   1945.2260.     -2.9032       3.7388
17.   2154.3140.     -1.8317       4.0503
18.   2368.1580.     -0.7033       4.2209
19.   2587.2330.     -2.4766       4.2500
20.   2811.6450.     -1.1301       4.2500
21.   3041.5000.      0.2490       4.2500
22.   3276.9040.      1.6614       4.2500
23.   3517.9660.      3.1078       4.2500
24